# 🚀 DEMO HỆ THỐNG DỊCH TỰ ĐỘNG: GIỌNG NÓI/VĂN BẢN SANG NGÔN NGỮ KÝ HIỆU VIỆT NAM (VSL)

**Báo cáo Tốt nghiệp - Khoa Thương mại Điện tử**

**Trường Đại học Kinh tế - Đại học Đà Nẵng**

---

### 📖 Giới thiệu chung
Notebook này là mã nguồn triển khai (Deployment) cho hệ thống hỗ trợ giao tiếp hai chiều dành cho người khiếm thính. Hệ thống thực hiện chuyển đổi đầu vào là **Giọng nói** hoặc **Văn bản Tiếng Việt** sang dạng **Cú pháp Ký hiệu (Gloss)** của Ngôn ngữ Ký hiệu Việt Nam.

### 🛠️ Kiến trúc kỹ thuật
Hệ thống tích hợp hai mô hình học sâu (Deep Learning) tiên tiến:
1.  **PhoWhisper (VinAI):** Module nhận dạng giọng nói (Automatic Speech Recognition - ASR), chuyển đổi âm thanh sang văn bản Tiếng Việt với độ chính xác cao.
2.  **ViT5 (Fine-tuned):** Module dịch máy (Neural Machine Translation), chuyển đổi văn bản Tiếng Việt sang chuỗi Gloss VSL.

### 📝 Hướng dẫn sử dụng
1.  Chạy toàn bộ các cell bên dưới (`Runtime` -> `Run all`).
2.  Đợi hệ thống cài đặt thư viện và tải Model (khoảng 1-2 phút).
3.  Một giao diện web (Gradio) sẽ hiện ra ở cuối trang. Bạn có thể:
    * Thu âm trực tiếp hoặc tải file âm thanh lên.
    * Nhập văn bản tiếng Việt để test khả năng dịch.

### Thiết lập thư viện & Môi trường

In [ ]:
# Cài đặt thư viện
!pip install -q gradio transformers torch librosa soundfile sentencepiece
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Import thư viện
import torch
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Đang sử dụng thiết bị: {DEVICE}")

Đang sử dụng thiết bị: cuda


### Tải model nhận dạng giọng nói tiếng Việt.

In [ ]:
# 1. Load PhoWhisper (Nhận dạng giọng nói tiếng Việt)
print("Đang tải model PhoWhisper...")
# Sử dụng pipeline của HuggingFace cho gọn
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="vinai/PhoWhisper-small",
    device=0 if torch.cuda.is_available() else -1
)

Đang tải model PhoWhisper...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

Device set to use cuda:0


### Tải model dịch thuật từ Google Drive

In [ ]:
# 2. Load Model Dịch (ViT5 - Model)
TRANSLATION_MODEL_PATH = "/content/drive/MyDrive/DUE - HỌC TẬP/YEAR 4 (2025-2026)/TTTN/Models/ViT5-base/trained_model_augmented_coffee"

In [ ]:
print(f"Đang tải model dịch từ: {TRANSLATION_MODEL_PATH}")
try:
    trans_tokenizer = AutoTokenizer.from_pretrained(TRANSLATION_MODEL_PATH)
    trans_model = AutoModelForSeq2SeqLM.from_pretrained(TRANSLATION_MODEL_PATH).to(DEVICE)
    print("Load model dịch thành công!")
except Exception as e:
    print(f"Lỗi load model dịch: {e}")
    print("Đang sử dụng chế độ demo (không có model dịch thực tế)")
    trans_model = None

Đang tải model dịch từ: /content/drive/MyDrive/DUE - HỌC TẬP/YEAR 4 (2025-2026)/TTTN/Models/ViT5-base/trained_model_augmented_coffee
Load model dịch thành công!


### Các hàm xử lý logic

In [ ]:
def speech_to_text(audio_path):
    """Chuyển đổi âm thanh sang văn bản tiếng Việt"""
    if audio_path is None:
        return ""
    try:
        # PhoWhisper xử lý
        output = asr_pipeline(audio_path)
        text = output['text']
        return text
    except Exception as e:
        return f"Lỗi nhận dạng: {str(e)}"

In [ ]:
def text_to_gloss(vietnamese_text):
    """Dịch tiếng Việt sang Gloss VSL"""
    if not vietnamese_text:
        return ""

    if trans_model is None:
        return "Chưa load được model dịch. Vui lòng kiểm tra đường dẫn."

    # Tiền xử lý: Thêm prefix giống lúc train
    input_text = f"vi: {vietnamese_text}"

    # Tokenize
    inputs = trans_tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(DEVICE)

    # Generate
    with torch.no_grad():
        outputs = trans_model.generate(
            inputs["input_ids"],
            max_length=128,
            num_beams=5,
            early_stopping=True
        )

    # Decode
    gloss_text = trans_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Hậu xử lý: Xóa prefix 'vsl: ' nếu model sinh ra
    if gloss_text.startswith("vsl: "):
        gloss_text = gloss_text.replace("vsl: ", "")

    return gloss_text

In [ ]:
def full_pipeline(audio, text_input, mode):
    """Luồng xử lý chính: Audio/Text -> Text TV -> Gloss"""

    vietnamese_output = ""

    # Xử lý đầu vào tùy theo mode
    if mode == "Giọng nói (Microphone/File)" and audio is not None:
        vietnamese_output = speech_to_text(audio)
    elif mode == "Văn bản (Nhập tay)" and text_input:
        vietnamese_output = text_input
    else:
        return "Vui lòng nhập dữ liệu đầu vào.", ""

    # Dịch sang Gloss
    gloss_output = text_to_gloss(vietnamese_output)

    return vietnamese_output, gloss_output

In [ ]:
print("Hoàn tất chuẩn bị!")

Hoàn tất chuẩn bị!


### Giao diện Demo

In [ ]:
# CSS tùy chỉnh để làm đẹp giao diện
custom_css = """
.container {max-width: 1200px; margin: auto; padding-top: 20px}
.header-text {text-align: center; font-family: 'Arial', sans-serif;}
.uni-name {font-size: 24px; font-weight: bold; color: #003366; margin-bottom: 5px;}
.faculty-name {font-size: 18px; font-weight: normal; color: #cc0000; margin-bottom: 20px;}
.project-title {font-size: 28px; font-weight: bold; color: #2c3e50; margin-bottom: 10px; border-bottom: 2px solid #eee; padding-bottom: 10px;}
.note-text {font-size: 14px; font-style: italic; color: #7f8c8d; margin-top: 20px; border-top: 1px solid #eee; padding-top: 10px;}
.output-box {border: 1px solid #e0e0e0; background-color: #f9f9f9; border-radius: 8px;}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    # --- HEADER ---
    with gr.Column(elem_classes="header-text"):
        gr.Markdown(
            """
            <div class="uni-name">TRƯỜNG ĐẠI HỌC KINH TẾ - ĐẠI HỌC ĐÀ NẴNG</div>
            <div class="faculty-name">KHOA THƯƠNG MẠI ĐIỆN TỬ</div>
            <div class="project-title">HỆ THỐNG NHẬN DẠNG GIỌNG NÓI VÀ DỊCH TỰ ĐỘNG NGÔN NGỮ KÝ HIỆU VIỆT NAM<br>NHẰM HỖ TRỢ GIAO TIẾP NGƯỜI KHIẾM THÍNH</div>
            """
        )

    # --- MAIN CONTENT ---
    with gr.Row():

        # CỘT TRÁI: INPUT
        with gr.Column(scale=1):
            gr.Markdown("### 1. Dữ liệu đầu vào (Input)")

            # Tab lựa chọn đầu vào
            with gr.Tabs():
                with gr.TabItem("Giọng nói (Microphone/File)"):
                    input_audio = gr.Audio(
                        sources=["microphone", "upload"],
                        type="filepath",
                        label="Thu âm hoặc Tải file"
                    )
                    mode_audio = gr.State(value="Giọng nói (Microphone/File)")
                    btn_audio = gr.Button("🚀 Xử lý Giọng nói", variant="primary")

                with gr.TabItem("Văn bản (Nhập tay)"):
                    input_text = gr.Textbox(
                        label="Nhập câu tiếng Việt",
                        placeholder="Ví dụ: Tôi muốn uống cà phê sữa.",
                        lines=3
                    )
                    mode_text = gr.State(value="Văn bản (Nhập tay)")
                    btn_text = gr.Button("🚀 Dịch Văn bản", variant="primary")

        # CỘT PHẢI: OUTPUT
        with gr.Column(scale=1):
            gr.Markdown("### 2. Kết quả xử lý (Output)")

            with gr.Group():
                gr.Markdown("#### Bước trung gian: Nhận dạng Tiếng Việt")
                output_vi = gr.Textbox(
                    label="Văn bản Tiếng Việt",
                    interactive=False,
                    show_copy_button=True,
                    elem_classes="output-box"
                )

                gr.Markdown("#### Kết quả cuối cùng: Cú pháp Ngôn ngữ ký hiệu (Gloss)")
                output_gloss = gr.Textbox(
                    label="VSL Gloss",
                    interactive=False,
                    show_copy_button=True,
                    elem_classes="output-box",
                    lines=3
                )

    # --- FOOTER / NOTE ---
    gr.Markdown(
        """
        <div class="note-text">
        ⚠️ <b>Lưu ý:</b> Phiên bản Demo hiện tại tập trung vào xử lý Ngôn ngữ tự nhiên (NLP).
        Việc phát triển thành các video ngắn, mô hình diễn họa 3D/Avatar chuyển động sẽ được tiếp tục nghiên cứu và tích hợp trong tương lai.
        </div>
        """
    )

    # --- SỰ KIỆN (ACTIONS) ---
    # Xử lý khi bấm nút Audio
    btn_audio.click(
        fn=full_pipeline,
        inputs=[input_audio, input_text, mode_audio], # input_text bị lờ đi trong mode này
        outputs=[output_vi, output_gloss]
    )

    # Xử lý khi bấm nút Text
    btn_text.click(
        fn=full_pipeline,
        inputs=[input_audio, input_text, mode_text], # input_audio bị lờ đi trong mode này
        outputs=[output_vi, output_gloss]
    )

In [ ]:
# Chạy ứng dụng
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2df4e003f7ab26ef80.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2df4e003f7ab26ef80.gradio.live
